In [1]:
import json
import igraph
import os
import re
import time
import logging
import sqlite3
import pandas as pd
import numpy as np
from lxml import objectify

In [2]:
# structure is this:
# -congress term
#     -votes
#          -year (2 possible)
#               -billno (h31 for house, s31 for senate)
#                    *data.json
#                         result: "Passed"
#                         bill:
#                              congress:
#                              number:
#                              type:
#                         date:
#                         requires: "1/2"
#                         category:
#                         session: yyyy
#                         type: ?
#                         subject: short description
#                         number: ?
#                         votes:
#                              Not_Voting: [{display_name: , party: , ID: , state: }]
#                              Nay: ...
#                              Present: ...
#                              Yay: ...
#                         chamber: 'h' or 's'
#                         congress: 104
#                         question: short description again
#                         vote_id: 'h10-104.1995'

In [2]:
data_dir = "/home/matt/Datasets/Congress/"
os.chdir(data_dir)

# Create and init the db

In [3]:
output_db = 'congress.db'
con = sqlite3.connect(output_db)
cur = con.cursor()

In [5]:
bill_schema = ['location', 'TEXT',
               'session', 'INTEGER',
               'roll', 'INTEGER',
               'year', 'INTEGER',
               'quarter', 'INTEGER',
               'month', 'INTEGER',
               'aye', 'INTEGER',
               'nay', 'INTEGER',
               'present', 'INTEGER',
               'nv','INTEGER',
               'required','TEXT',
               'result','TEXT',
               'category','TEXT',
               'type','TEXT',
               'question','TEXT']
bill_fields = bill_schema[0::2]
bill_keys = bill_fields[0:4]
print(bill_fields)
print(bill_keys)

['location', 'session', 'roll', 'year', 'quarter', 'month', 'aye', 'nay', 'present', 'nv', 'required', 'result', 'category', 'type', 'question']
['location', 'session', 'roll', 'year']


In [6]:
vote_schema = ['location','TEXT',
               'session','INTEGER',
               'roll','INTEGER',
               'year','INTEGER',
               'quarter', 'INTEGER',
               'month', 'INTEGER',
               'id','TEXT',
               'vote','TEXT',
               'value','TEXT',
               'state','TEXT']
vote_fields = vote_schema[0::2]
vote_keys = vote_fields[0:4] + vote_fields[6:7]
print(vote_fields)
print(vote_keys)

['location', 'session', 'roll', 'year', 'quarter', 'month', 'id', 'vote', 'value', 'state']
['location', 'session', 'roll', 'year', 'id']


In [7]:
bills_command = (
   "create table bills ("+
    "{} {}, "*len(bill_fields)+
    "PRIMARY KEY ("+
    ','.join(bill_keys)+
    ") )").format(*bill_schema)
votes_command = (
   "create table votes ("+
    "{} {}, "*len(vote_fields)+
    "PRIMARY KEY ("+
    ','.join(vote_keys)+
    ") )").format(*vote_schema)

In [8]:
print(votes_command)
print(bills_command)

create table votes (location TEXT, session INTEGER, roll INTEGER, year INTEGER, quarter INTEGER, month INTEGER, id TEXT, vote TEXT, value TEXT, state TEXT, PRIMARY KEY (location,session,roll,year,id) )
create table bills (location TEXT, session INTEGER, roll INTEGER, year INTEGER, quarter INTEGER, month INTEGER, aye INTEGER, nay INTEGER, present INTEGER, nv INTEGER, required TEXT, result TEXT, category TEXT, type TEXT, question TEXT, PRIMARY KEY (location,session,roll,year) )


In [9]:
cur.execute(votes_command)
cur.execute(bills_command)
con.commit()

# Read the data to the db

In [10]:
# all the data will come from here
os.chdir('congress')

### regexes we'll need

In [11]:
congress_num = re.compile('[0-9]{1,3}')
congress_bill = re.compile('(s|h)[0-9]{1,5}')
date_regex = re.compile(r'([0-9]{4}(-|/)[0-9]{1,2}\2[0-9]{1,2})')

### functions we'll need

In [17]:
def insert_rows(cur,table,data,fields,how='ignore'):
    command = ("insert or {} into {} VALUES ("+','.join(["?"]*len(fields))+")").format(how,table)
    cur.executemany(command,data)

def get_bill_data(root):
    datetime = pd.to_datetime(root.attrib['datetime'])
    year = int(datetime.year)
    quarter = 4*year+int(datetime.quarter)
    month = 12*year+int(datetime.month)
    
    counts = [int(get_attr(root,kind,0)) for kind in ['aye','nay','present','nv']]
    text_attrs = [get_node_text(root,tag,'') for tag in ['required','result','category','type','question']]
    
    return [root.attrib['where'],int(root.attrib['session']),int(root.attrib['roll']),
            year, quarter, month] + counts + text_attrs
    
def get_attr(obj,attr,default):
    return obj.attrib.get(attr,default)

def get_node_text(obj,tag,default):
    return obj.__dict__.get(tag,objectify.StringElement(default)).text
    

### This shows that every bill has an xml; we'll use these because the json is a mess

In [14]:
congresses = sorted([name for name in os.listdir() if re.match(congress_num,name)],key=lambda s: int(s))

# should be no output if every bill has an xml file
for congress in congresses:
    vote_dir = os.path.join(congress,'votes')
    years = sorted(os.listdir(vote_dir))
    
    for year in years:
        bills = os.listdir(os.path.join(vote_dir,year))
        bills = sorted([num for num in bills if re.match(congress_bill,num)])
        
        for bill in bills:
            bill_dir = os.path.join(vote_dir,year,bill)
            files = os.listdir(bill_dir)
            files = [ file for file in files if file.endswith('.xml')]
            if len(files)==0:
                print(bill_dir)

## Loop through the bills and store them with the votes

In [18]:
congresses = sorted([name for name in os.listdir() if re.match(congress_num,name)],key=lambda s: int(s))

#c=0
# for every session
for congress in congresses:
#     c+=1
#     if c>2:
#         break
    
    vote_dir = os.path.join(congress,'votes')
    years = sorted(os.listdir(vote_dir))
    
    # for every year of that session
    for year in years:
        bills = os.listdir(os.path.join(vote_dir,year))
        bills = sorted([num for num in bills if re.match(congress_bill,num)], 
                       key = lambda s: int(s[1:]))
        
        bill_data = []
        # for every bill in that year
        for bill in bills:
            # list the files in the bill dir and parse the xml
            bill_dir = os.path.join(vote_dir,year,bill)
            files = os.listdir(bill_dir)
            xmlfiles = [file for file in files if file.endswith('.xml')]
            
            with open(os.path.join(vote_dir,year,bill,xmlfiles[0]), 'r') as infile:
                # get the xml object representing all the votes
                data = objectify.parse(infile).getroot()
                
            # get the bill data from the xml object
            new_bill = get_bill_data(data)
            
            # append to the list for that year
            bill_data.append(tuple(new_bill))
            
            # get the vote data from the xml object
            voters = data.voter
            vote_data = [tuple(new_bill[0:6] + [voter.attrib.get(attr,None) for attr in vote_fields[6:]]) for voter in voters]
            
            if np.random.rand() > 0.9997:
                print(new_bill)
                print(vote_data[np.random.randint(0,len(vote_data))])
            
            # insert the vote data into the db for each bill
            insert_rows(cur,'votes',vote_data,vote_fields,'ignore')
        
        # insert the bill data into the db for each year
        insert_rows(cur,'bills',bill_data,bill_fields,'ignore')
        
        # commit the changes
        con.commit()
        
    print("congress {} completed".format(congress))

congress 1 completed
congress 2 completed
congress 3 completed
congress 4 completed
congress 5 completed
congress 6 completed
congress 7 completed
congress 8 completed
congress 9 completed
congress 10 completed
['senate', 11, 126, 1811, 7245, 21733, 23, 7, 0, 4, 'unknown', 'unknown', 'unknown', 'On Passage', 'TO PASS THE SENATE BILL, AS AMENDED, AND APPROPRIATE $100,000 THEREFORE.']
('senate', 11, 126, 1811, 7245, 21733, '411899', '+', 'Yea', 'OH')
congress 11 completed
congress 12 completed
congress 13 completed
congress 14 completed
congress 15 completed
congress 16 completed
congress 17 completed
congress 18 completed
congress 19 completed
congress 20 completed
congress 21 completed
congress 22 completed
['house', 23, 27, 1834, 7337, 22010, 107, 88, 0, 41, 'unknown', 'unknown', 'unknown', 'TO TABLE THE RESOLUTION PROVIDING THAT THE COMMITTEE ON INDIAN AFFAIRS BE INSTRUCTED TO INQUIRE WHETHER THE PROVISIONS OF THE TREATY OF MARCH 1832, WITH THE CREEK TRIBE OF INDIANS, IN ALABAMA, BE 

In [19]:
# check the schema
cur.execute("select * from sqlite_master").fetchall()

[('table',
  'votes',
  'votes',
  2,
  'CREATE TABLE votes (location TEXT, session INTEGER, roll INTEGER, year INTEGER, quarter INTEGER, month INTEGER, id TEXT, vote TEXT, value TEXT, state TEXT, PRIMARY KEY (location,session,roll,year,id) )'),
 ('index', 'sqlite_autoindex_votes_1', 'votes', 3, None),
 ('table',
  'bills',
  'bills',
  4,
  'CREATE TABLE bills (location TEXT, session INTEGER, roll INTEGER, year INTEGER, quarter INTEGER, month INTEGER, aye INTEGER, nay INTEGER, present INTEGER, nv INTEGER, required TEXT, result TEXT, category TEXT, type TEXT, question TEXT, PRIMARY KEY (location,session,roll,year) )'),
 ('index', 'sqlite_autoindex_bills_1', 'bills', 5, None)]

In [20]:
print(vote_fields)
print(bill_fields)

['location', 'session', 'roll', 'year', 'quarter', 'month', 'id', 'vote', 'value', 'state']
['location', 'session', 'roll', 'year', 'quarter', 'month', 'aye', 'nay', 'present', 'nv', 'required', 'result', 'category', 'type', 'question']


In [21]:
# from the last bill
vote_data[0:10]

[('house', 114, 163, 2016, 8066, 24196, '412630', '+', 'Yea', 'LA'),
 ('house', 114, 163, 2016, 8066, 24196, '400004', '+', 'Yea', 'AL'),
 ('house', 114, 163, 2016, 8066, 24196, '412615', '+', 'Yea', 'CA'),
 ('house', 114, 163, 2016, 8066, 24196, '412625', '+', 'Yea', 'GA'),
 ('house', 114, 163, 2016, 8066, 24196, '412438', '+', 'Yea', 'MI'),
 ('house', 114, 163, 2016, 8066, 24196, '412642', '+', 'Yea', 'NE'),
 ('house', 114, 163, 2016, 8066, 24196, '412655', '+', 'Yea', 'TX'),
 ('house', 114, 163, 2016, 8066, 24196, '412469', '+', 'Yea', 'PA'),
 ('house', 114, 163, 2016, 8066, 24196, '412541', '+', 'Yea', 'KY'),
 ('house', 114, 163, 2016, 8066, 24196, '400018', '+', 'Yea', 'TX')]

In [24]:
# from the db
cur.execute("select * from votes where session = 114 and location = 'house' and year = 2016 and roll = 163 limit 10").fetchall()

[('house', 114, 163, 2016, 8066, 24196, '400004', '+', 'Yea', 'AL'),
 ('house', 114, 163, 2016, 8066, 24196, '400018', '+', 'Yea', 'TX'),
 ('house', 114, 163, 2016, 8066, 24196, '400021', '-', 'Nay', 'CA'),
 ('house', 114, 163, 2016, 8066, 24196, '400029', '+', 'Yea', 'UT'),
 ('house', 114, 163, 2016, 8066, 24196, '400030', '+', 'Yea', 'GA'),
 ('house', 114, 163, 2016, 8066, 24196, '400032', '+', 'Yea', 'TN'),
 ('house', 114, 163, 2016, 8066, 24196, '400033', '+', 'Yea', 'OR'),
 ('house', 114, 163, 2016, 8066, 24196, '400046', '+', 'Yea', 'TX'),
 ('house', 114, 163, 2016, 8066, 24196, '400047', '-', 'Nay', 'PA'),
 ('house', 114, 163, 2016, 8066, 24196, '400048', '-', 'Nay', 'FL')]

In [25]:
len(cur.execute("select id from votes").fetchall())
# 23,602,445 votes

23602445

# Now build the senators table

### import the data

In [26]:
os.chdir('..')
os.chdir('congress-legislators')

historic_senators = pd.read_csv('legislators-historic.csv')
current_senators = pd.read_csv('legislators-current.csv')

print(historic_senators.shape)
print(current_senators.shape)

print(historic_senators.columns)
print(current_senators.columns)

(11807, 29)
(540, 29)
Index(['last_name', 'first_name', 'birthday', 'gender', 'type', 'state',
       'district', 'party', 'url', 'address', 'phone', 'contact_form',
       'rss_url', 'twitter', 'facebook', 'facebook_id', 'youtube',
       'youtube_id', 'bioguide_id', 'thomas_id', 'opensecrets_id', 'lis_id',
       'cspan_id', 'govtrack_id', 'votesmart_id', 'ballotpedia_id',
       'washington_post_id', 'icpsr_id', 'wikipedia_id'],
      dtype='object')
Index(['last_name', 'first_name', 'birthday', 'gender', 'type', 'state',
       'district', 'party', 'url', 'address', 'phone', 'contact_form',
       'rss_url', 'twitter', 'facebook', 'facebook_id', 'youtube',
       'youtube_id', 'bioguide_id', 'thomas_id', 'opensecrets_id', 'lis_id',
       'cspan_id', 'govtrack_id', 'votesmart_id', 'ballotpedia_id',
       'washington_post_id', 'icpsr_id', 'wikipedia_id'],
      dtype='object')


### combine the historic and current data and add a few columns

In [27]:
current_senators.set_index('govtrack_id',verify_integrity=True,inplace=True)
historic_senators.set_index('govtrack_id',verify_integrity=True,inplace=True)

senators = pd.concat([historic_senators,current_senators],axis=0,join='outer',verify_integrity=True)
senators.shape

(12347, 28)

In [28]:
def year_quarter_month(datetime):
    dt = pd.to_datetime(datetime)
    return pd.Series([dt.year,4*dt.year+dt.quarter,12*dt.year+dt.month])

In [29]:
df = senators.birthday.apply(year_quarter_month)
df.columns = pd.Index(['birth_year','birth_quarter','birth_month'])

df.shape

(12347, 3)

In [30]:
senators = pd.concat([senators,df],axis=1,join='outer')
senators.head(5)

,last_name,first_name,birthday,gender,type,state,district,party,url,address,...,lis_id,cspan_id,votesmart_id,ballotpedia_id,washington_post_id,icpsr_id,wikipedia_id,birth_year,birth_quarter,birth_month
govtrack_id,,,,,,,,,,,,,,,,,,,,,
401222,Bassett,Richard,1745-04-02,M,sen,DE,NaN,Anti-Administration,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,507,NaN,1745,6982,20944
401521,Bland,Theodorick,1742-03-21,M,rep,VA,9,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,786,NaN,1742,6969,20907
402032,Burke,Aedanus,1743-06-16,M,rep,SC,2,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1260,Aedanus Burke,1743,6974,20922
402334,Carroll,Daniel,1730-07-22,M,rep,MD,6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1538,Daniel Carroll,1730,6923,20767
402671,Clymer,George,1739-03-16,M,rep,PA,-1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1859,George Clymer,1739,6957,20871


In [31]:
senators.shape

(12347, 31)

### The database schema and create command

In [32]:
senator_schema = ['id','TEXT','first_name','TEXT','last_name','TEXT',
                  'state','TEXT','district','TEXT','party','TEXT',
                  'birthday','TEXT','birth_year','INTEGER','birth_quarter','INTEGER','birth_month','INTEGER',
                  'url','TEXT','twitter','TEXT','facebook','TEXT','facebook_id','TEXT',
                  'youtube','TEXT','youtube_id','TEXT',
                  'opensecrets_id','TEXT','wikipedia_id','TEXT','washington_post_id','TEXT']
senator_fields = senator_schema[0::2]
senator_keys = senator_schema[0:1]
senator_types = senator_schema[1::2]
print(senator_fields)
print(senator_keys)
print(senator_types)

['id', 'first_name', 'last_name', 'state', 'district', 'party', 'birthday', 'birth_year', 'birth_quarter', 'birth_month', 'url', 'twitter', 'facebook', 'facebook_id', 'youtube', 'youtube_id', 'opensecrets_id', 'wikipedia_id', 'washington_post_id']
['id']
['TEXT', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'INTEGER', 'INTEGER', 'INTEGER', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'TEXT', 'TEXT']


In [33]:
senators_command = (
    "create table senators ("+
    "{} {}, "*len(senator_fields)+
    "PRIMARY KEY ("+
    ','.join(senator_keys)+
    ") )").format(*senator_schema)
print(senators_command)

create table senators (id TEXT, first_name TEXT, last_name TEXT, state TEXT, district TEXT, party TEXT, birthday TEXT, birth_year INTEGER, birth_quarter INTEGER, birth_month INTEGER, url TEXT, twitter TEXT, facebook TEXT, facebook_id TEXT, youtube TEXT, youtube_id TEXT, opensecrets_id TEXT, wikipedia_id TEXT, washington_post_id TEXT, PRIMARY KEY (id) )


### Subset on the columns we want in the db

In [34]:
senators['id'] = senators.index
print(senators.shape)
print(senators.columns)

(12347, 32)
Index(['last_name', 'first_name', 'birthday', 'gender', 'type', 'state',
       'district', 'party', 'url', 'address', 'phone', 'contact_form',
       'rss_url', 'twitter', 'facebook', 'facebook_id', 'youtube',
       'youtube_id', 'bioguide_id', 'thomas_id', 'opensecrets_id', 'lis_id',
       'cspan_id', 'votesmart_id', 'ballotpedia_id', 'washington_post_id',
       'icpsr_id', 'wikipedia_id', 'birth_year', 'birth_quarter',
       'birth_month', 'id'],
      dtype='object')


In [35]:
# The columns we want to keep
senators = senators[senator_fields]

### Put it all in the db

In [36]:
def type_or_null(i,datatype):
    if pd.isnull(i):
        return None
    else:
        return datatype(i)

typedict = {'TEXT':str,'INTEGER':int}

In [37]:
data = []
for index in senators.index:
    row = senators.loc[index,]
    row_data = []
    for i in range(len(row)):
        row_data.append(type_or_null(row.iloc[i],typedict[senator_types[i]]))
    data.append(tuple(row_data))

In [38]:
print(len(data))
print(len(data[0]))
print(len(senator_fields))

12347
19
19


In [39]:
cur.execute(senators_command)
con.commit()

insert_rows(cur,'senators',data,senator_fields)
con.commit()

In [40]:
print(len(cur.execute('select youtube from senators').fetchall()))

12347


In [41]:
# every year is there- no gaps bigger than 1
np.diff(np.array(sorted(list(zip(*cur.execute("select distinct year from votes").fetchall()))[0])))

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

# Create indices for fast future queries

In [ ]:
# indices for fast queries
for time_col in ['year','quarter','month']:
    cur.execute("create index vote_{0} on votes({0},location)".format(time_col))
    con.commit()

# That's all folks!

In [42]:
con.close()